## Silver: Exportação Vinhos

In [0]:
%run "../1 - Setup/Ingestao_bibliotecas_padrao"


In [0]:
from pyspark.sql.functions import col

In [0]:

import re

# 1️⃣ Lê a tabela do Spark e converte para Pandas
df_original = spark.table("vitivinicultura.bronze_exportacao.exportacao_vinhos").toPandas()

# 2️⃣ Define a coluna com os dados separados por ";"
coluna = "_c0"

# 3️⃣ Colunas adicionais
colunas_adicionais = ["ingestao_ts", "usuario_ingestao"]
df_extra = df_original[colunas_adicionais].copy()

# 4️⃣ Quebra a string em colunas separadas
linhas = df_original[coluna].str.split(';', expand=True)

# 5️⃣ Extrai cabeçalho da primeira linha
cabecalho = linhas.iloc[0].astype(str).fillna("coluna_sem_nome").tolist()
linhas.columns = cabecalho

# 6️⃣ Remove a primeira linha (cabeçalho original)
df_tratado = linhas.iloc[1:].reset_index(drop=True)

# 7️⃣ Ajusta nomes duplicados com regra especial para anos
contador_anos = {}
novos_nomes = []

for c in linhas.columns:
    # Se for um número de 4 dígitos (ano)
    if re.match(r"^\d{4}$", c):
        contador_anos[c] = contador_anos.get(c, 0) + 1
        if contador_anos[c] == 1:
            novos_nomes.append(f"{c}_litros")
        elif contador_anos[c] == 2:
            novos_nomes.append(f"{c}_usd")
        else:
            novos_nomes.append(f"{c}_{contador_anos[c]}")
    else:
        # Evita duplicação de outros nomes
        if c in novos_nomes:
            qtd = sum(1 for x in novos_nomes if x.startswith(c))
            novos_nomes.append(f"{c}_{qtd+1}")
        else:
            novos_nomes.append(c)

df_tratado.columns = novos_nomes

# 8️⃣ Converte todas as colunas para string e preenche valores nulos
for c in df_tratado.columns:
    df_tratado[c] = df_tratado[c].astype(str).fillna("")

# 9️⃣ Junta novamente com as colunas adicionais
df_final = pd.concat([df_extra.reset_index(drop=True), df_tratado], axis=1)

# 🔟 Remove linhas com Id nulo, se existir a coluna
if "Id" in df_final.columns:
    df_final = df_final[df_final["Id"].notnull()]

# 11️⃣ Reordena colunas para deixar anos em ordem crescente
anos_cols = sorted([c for c in df_final.columns if re.match(r"^\d{4}_(litros|usd)$", c)])
outras_cols = [c for c in df_final.columns if c not in anos_cols]
df_final = df_final[outras_cols + anos_cols]

# 🔹 Visualiza o resultado final
display(df_final)

In [0]:
 #Cria tabela gerenciada no Unity Catalog
spark.sql("""
CREATE TABLE IF NOT EXISTS vitivinicultura.silver_exportacao.exportacao_vinhos
USING DELTA
""")
df_spark = spark.createDataFrame(df_final)
# Escrita no formato Delta Lake
df_spark.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("vitivinicultura.silver_exportacao.exportacao_vinhos")

